# Regresión Lineal Múltiple

Vamos a continuar con el capítulo 3 del libro ["Introduction to Statistical Learning"](http://www-bcf.usc.edu/~gareth/ISL/), ahora con la sección 3.2

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns

from sklearn.preprocessing import scale
import sklearn.linear_model as skl_lm
import sklearn.metrics
import statsmodels.api as sm
import statsmodels.formula.api as smf




## Cargamos los datos

Cargamos los conjuntos de datos que vamos a usar

Los conjuntos de datos están en la web del libro (pero ya están descargados)
http://www-bcf.usc.edu/~gareth/ISL/data.html

In [2]:
advertising = pd.read_csv('advertising.csv', usecols=[1,2,3,4], na_values='?').dropna()  
# na_values='?').dropna()  borra las filas y ciolumnas que tienen Null/NaN 
advertising.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   TV         200 non-null    float64
 1   Radio      200 non-null    float64
 2   Newspaper  200 non-null    float64
 3   Sales      200 non-null    float64
dtypes: float64(4)
memory usage: 7.8 KB


## Regresión y ajuste de modelos

El análisis de regresión consiste en encontrar un  **modelo** que relaciona los valores medidos de una variable **objetivo** (tb se llama la **respuesta**) en función de un conjunto de variables **explicativas** (tb **variables predictoras**, o **variables explicativas**, o **regresores**).

Los valores medidos en el mundo real nunca se ajustan de forma perfecta a un modelo, debido en primer lugar a errores de medida, pero también a que cualquier modelo matemático es una *simplificación* del mundo real, y si tuviera en cuenta todos los factores que influyen en un conjunto de variables, sería inmanejable.

Por tanto, no tiene sentido aspirar a encontrar un modelo que prediga exactamente los valores medidos, y debemos admitir que el modelo cometerá un cierto error.

Un modelo útil encuentra una relación funcional sencilla en conjuntos de pocas variables. Se trata de explicar una variable objetivo en función de otro conjunto de variables mejor conocidas o más fáciles de medir. El  **análisis de regresión**  (más exactamente, el análisis de regresión  *paramétrico*) permite encontrar un modelo explicativo en dos etapas:


 1. Nuestro conocimiento del tema en cuestión nos permite escribir un modelo que afirma que la variable  *Y*  es una función de las variables $X_1,\dots,X_p$. La variable  *Y* se suele llamar la **respuesta** y las variables  $X_1,\dots,X_p$ se llaman  **variables predictoras**. La forma exacta de la función no está fijada a priori, sino que depende de unos pocos  **parámetros**  libres.
 
 Por ejemplo, para la **regresión lineal**, el modelo es
 $$
 Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \dots + \beta_p X_p + \epsilon
 $$
 donde $\beta_0,\dots,\beta_p$ son los parámetros y $\epsilon$ es un error que no podemos explicar dentro de este modelo.
 
 2. **Ajustamos el modelo** a los datos de que disponemos, eligiendo los valores de los parámetros para los que la distancia entre los valores medidos de la variable  *Y*  y los valores predichos aplicando el modelo minimizan el error cometido. El error que se suele minimizar es el error cuadrático (**residual sum of squares**):
$$
RSS = e_1^2 + e_2^2 + \dots + e_n^2
$$
donde
$$
e_1 = y_1 - (\beta_0 + \beta_1 X_1^1 + ... + \beta_p X_p^1), \dots e_n = y_n - (\beta_0 + \beta_1 X_1^n  + ... + \beta_p X_p^n).
$$
Es decir, entre todas las posibles combinaciones de coeficientes $(\beta_0,\dots,\beta_p)$ nos quedamos con aquella combinación $(\hat{\beta_0},\hat{\beta_1},\dots,\hat{\beta_1})$ que minimiza el RSS, y hemos obtenido la  **regresión lineal por mínimos cuadrados**

## Ejemplo: Inversión en publicidad

Hacer regresión múltiple consiste en encontrar una función lineal de las variables predictoras que aproxima la función objetivo:

$$
Sales\approx \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper
$$

Si incorporamos el término de error, podemos sustituir el signo de aproximación $\approx$ por uno de igualdad:

$$
Sales = \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper + \varepsilon
$$


que minimiza el error cuadrático. Es cualitiva y cuantitivamente distinto de ajustar modelos por separado para cada variable predictora:

$$
Sales = \beta_0^{TV} + \beta_1^{TV}\times TV + \varepsilon
$$

$$
Sales = \beta_0^R + \beta_1^R\times radio  + \varepsilon
$$

$$
Sales = \beta_0^N + \beta_1^{N}\times newspaper + \varepsilon
$$

### Regresión lineal múltiple con scikit_learn

  1. Definimos un objeto de tipo "LinearRegression"

```python
regr = skl_lm.LinearRegression()
```

  2. El método ``fit`` **"ajusta"** la función lineal, encontrando los valores de $(\beta_0, \beta_1, \dots, \beta_p)$ para los que el error cuadrático cometido es menor. Necesita dos argumentos:
      - Un **array 2D de numpy** o un **DataFrame** X con las variables predictoras. No acepta una serie ni un array 1D.
      - Una **Serie** y con la variable objetivo.

```python
X = advertising[['TV', 'Radio', 'Newspaper']]
y = advertising.Sales

regr.fit(X,y)
```

y ya tenemos el objeto ``regr`` que contiene la recta ajustada por mínimos cuadrados.

In [3]:
regr = skl_lm.LinearRegression()

X = advertising[['TV', 'Radio', 'Newspaper']]
y = advertising.Sales

regr.fit(X,y)

print(regr.intercept_)  #beta0
print(regr.coef_)       #beta1 .. betap

2.938889369459412
[ 0.04576465  0.18853002 -0.00103749]


#### predecir 

Ahora que el modelo está ajustado, podemos predecir la cantidad de ventas, conocido el valor de las variables predictoras.

```python
regr.predict([[inv_TV, inv_Radio, inv_Newspaper]])
```

In [4]:
2.938889369459412 + 0.04576465*200 +  0.18853002*20 -0.00103749*20

15.84166996945941

In [5]:
#El orden de (inv_TV, inv_Radio, inv_Newspaper) es importante: 
#en el mismo orden que al llamar a regr.fit(...)
regr.predict([[200,20,20]])

array([15.84166894])

También podemos llamar a ``regr.predict()`` con un DataFrame como argumento, por ejemplo para predecir el nivel de ventas para distintas asignaciones alternativas de un presupuesto de 300k para inversión en publicidad.

**Atención**: es necesario que las columnas en el DataFrame aparezcan *en el mismo orden* que usamos al entrenar el modelo. Se debe a que ``scikit-learn`` fue concebida pensando en ``numpy``, antes de que ``pandas`` se hiciera tan popular.

In [6]:
advertising_future = pd.DataFrame({
        'TV':       [100,100,70 ,30 ,30 ,70 ],
        'Radio':    [30 ,70 ,30 ,70 ,100,100],
        'Newspaper':[70 ,30,100 ,100,70 ,30 ]
})
advertising_future


,TV,Radio,Newspaper
0,100,30,70
1,100,70,30
2,70,30,100
3,30,70,100
4,30,100,70
5,70,100,30


In [7]:
#añadimos una columna que sea el total
advertising_future['total']=advertising_future.TV + advertising_future.Radio + advertising_future.Newspaper

#hacemos la regresión multiple
advertising_future['predict scikit'] = regr.predict( advertising_future[['TV','Radio','Newspaper']] )
advertising_future.sort_values(by=['predict scikit'])

,TV,Radio,Newspaper,total,predict scikit
2,70,30,100,200,11.694566
0,100,30,70,200,13.098630
3,30,70,100,200,17.405181
1,100,70,30,200,20.681330
4,30,100,70,200,23.092206
5,70,100,30,200,24.964291


### Statsmodels 

Vamos a usar también la librería statsmodel, que imita la sintaxis de la regresión lineal en ``R``.

La fórmula ``Sales ~ TV + Radio`` significa que busca el modelo de regresión lineal:
$$
Sales\approx \beta_0 + \beta_1\times TV + \beta_2 \times Radio + \varepsilon
$$

 - Ajustamos el modelo $Sales\approx \beta_0 + \beta_1\times TV + \beta_2 \times Radio+ \varepsilon$ usando el DataFrame ``advertising``.
 
```python
recta = smf.ols('Sales ~ TV + Radio', advertising).fit()
```

In [8]:
stat = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()

In [9]:
advertising_future['predict statsmodel']=stat.predict(advertising_future)
advertising_future

,TV,Radio,Newspaper,total,predict scikit,predict statsmodel
0,100,30,70,200,13.098630,13.098630
1,100,70,30,200,20.681330,20.681330
2,70,30,100,200,11.694566,11.694566
3,30,70,100,200,17.405181,17.405181
4,30,100,70,200,23.092206,23.092206
5,70,100,30,200,24.964291,24.964291


### Interpretando el modelo ajustado

In [10]:
stat.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.9389,0.312,9.422,0.000,2.324,3.554
TV,0.0458,0.001,32.809,0.000,0.043,0.049
Radio,0.1885,0.009,21.893,0.000,0.172,0.206
Newspaper,-0.0010,0.006,-0.177,0.860,-0.013,0.011


  - En la tabla resumen de statsmodel, podemos observar dos columnas ``[0.025 	0.975]``, que delimitan un **intervalo de confianza al 95%** para cada coeficiente.Representan **un intervalo de _"valores razonables"_ para el valor de cada coeficiente** $\beta_0,\dots \beta_p$.
  - std err es una estimación del error estándar cometido en la estimación de cada coeficiente
  - t: el estadístico t es el ratio entre la estimación del coeficiente y el error estándar.
  - P>|t|: el p-valor asociado al contraste de hipótesis.


In [11]:
stat = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
stat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Sales   R-squared:                       0.897
Model:                            OLS   Adj. R-squared:                  0.896
Method:                 Least Squares   F-statistic:                     570.3
Date:                Mon, 10 Jan 2022   Prob (F-statistic):           1.58e-96
Time:                        17:05:35   Log-Likelihood:                -386.18
No. Observations:                 200   AIC:                             780.4
Df Residuals:                     196   BIC:                             793.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      2.9389      0.312      9.422      0.000       2.324       3.554
TV             0.0458      0.001     32.809      0.000       0.043       0.049
Radio          0.1885      0.009     21.893      0.000       0.172       0.206
Newspaper     -0.0010      0.006     -0.177      0.860      -0.013       0.011
==============================================================================
Omnibus:                       60.414   Durbin-Watson:                   2.084
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              151.241
Skew:                          -1.327   Prob(JB):                     1.44e-33
Kurtosis:                       6.332   Cond. No.                         454.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Si ajustamos el modelo lineal que depende sólo de Newspaper

$$
Sales\approx \beta_0 + \beta_3\times newspaper + \varepsilon,
$$

nos sale que la pendiente de la recta es positiva con un p-valor del 0.1%, pero si ajustamos el modelo lineal completo:

$$
Sales\approx \beta_0 + \beta_1\times TV  + \beta_2\times radio  + \beta_3\times newspaper + \varepsilon
$$

la pendiente $\beta_3$ es muy próxima a 0 (el p-valor es muy alto, el intervalo de confianza contiene a 0...)

In [12]:
stat = smf.ols('Sales ~ TV', advertising).fit()
stat.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,7.0326,0.458,15.360,0.000,6.130,7.935
TV,0.0475,0.003,17.668,0.000,0.042,0.053


In [13]:
stat = smf.ols('Sales ~ Radio', advertising).fit()
stat.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,9.3116,0.563,16.542,0.000,8.202,10.422
Radio,0.2025,0.020,9.921,0.000,0.162,0.243


In [14]:
stat = smf.ols('Sales ~ Newspaper', advertising).fit()
stat.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,12.3514,0.621,19.876,0.000,11.126,13.577
Newspaper,0.0547,0.017,3.300,0.001,0.022,0.087


En la tabla de resumen para regresión múltiple, aparece una fila para cada coeficiente del modelo lineal.

In [15]:
stat = smf.ols('Sales ~ TV + Radio + Newspaper', advertising).fit()
stat.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
Intercept,2.9389,0.312,9.422,0.000,2.324,3.554
TV,0.0458,0.001,32.809,0.000,0.043,0.049
Radio,0.1885,0.009,21.893,0.000,0.172,0.206
Newspaper,-0.0010,0.006,-0.177,0.860,-0.013,0.011


In [16]:
advertising[['TV', 'Radio', 'Newspaper']].corr()

,TV,Radio,Newspaper
TV,1.000000,0.054809,0.056648
Radio,0.054809,1.000000,0.354104
Newspaper,0.056648,0.354104,1.000000


¿Qué sentido tiene que el coeficiente de **newspaper** sea positivo en la regresión simple y cercano a 0 en la regresión múltiple? Se puede interpretar así:

> En aquellos países en los que se invierte más en anuncios en periódicos, también se suele invertir en TV y radio (lo comprobamos al observar que la correlación entre **newspaper** y **Radio** es del 35%). El modelo lineal múltiple indica que los anuncios en TV y radio son eficaces (aumentan las ventas), mientras que los anuncios en periódicos no. Sin embargo, si hacemos la regresión simple, resulta que los países en los que más se invierte más en anuncios en periódicos tienen más ventas que aquellos en los que se invierte menos, pero es debido a que también se invierte más en Radio, y no es causa directa de los anuncios en periódicos.

Un ejemplo clásico:
> Ajustamos un modelo lineal a $Y$ (ataques de tiburones) contra $X_1$ (ventas de helados en la playa) y encontramos una pendiente (correlación) positiva: a mayor ventas de helados, más ataques de tiburones. Obviamente no hay relación de causalidad entre ambas variables, pero en este caso podemos encontrar una causa que explica esta correlación: cuanta más gente en la playa ($X_2$), más ventas de helados y más ataques a tiburones. Una regresión múltiple $Y\sim X_1,X_2$, no muestra relación positiva entre $X_1$ e $Y$, y sí lo hace entre $X_2$ e $Y$.

Para más información, puedes leer sobre [factores de confusión](https://en.wikipedia.org/wiki/Confounding).

## ¿Cómo decidimos si un modelo es bueno?

Supongamos que el mejor modelo lineal que hemos encontrado es 
$$
y=f(\mathbf{x})+\epsilon=\beta_0 + x_1\beta_1 +\dots +x_p\beta_p+\epsilon,\qquad \mathbf{x}=(x_1,\dots,x_p).
$$

 - **TSS**: Total sum of squares: $\Sigma_i (y_i-\bar{y})^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la media $\bar{y}$)
 - **RSS**: Residual sum of squares:  $\Sigma_i (y_i-f(\mathbf{x}_i))^2$ (sumamos el cuadrado de la diferencia entre el dato $y_i$ y la predicción $f(\mathbf{x}_i)$ usando nuestro modelo)
 - **RSS/TSS**: cociente entre la "varianza residual" y la "varianza total"
 - **R-cuadrado**: "porcentaje de la varianza que el modelo explica". 
$$
R^2 = 1 - \frac{RSS}{TSS}
$$

```python
#Calcula R^2 con scikit-learn
regr.score(X,y)
```

```python
#Devuelve R^2 para el modelo ajustado con statsmodels
est.rsquared
```

In [17]:
#Calcula R^2 con scikit-learn
regr.score(X,y)

0.8972106381789522

In [18]:
#Devuelve R^2 para el modelo ajustado con statsmodels
stat.rsquared

0.8972106381789522